In [1]:
import glob
import json
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import re
import operator 
from collections import Counter
from nltk import bigrams
import pandas as pd
import csv
import unicodecsv
from os import listdir
from os.path import isfile, join
import glob
import wordcloud
import matplotlib.pyplot as plt
import numpy as np

In [2]:
path = "full_DataFrame/combined_data/texts_data/"
path

'full_DataFrame/combined_data/texts_data/'

In [3]:
df = pd.read_msgpack(path+"all_tweets.msg")
df.head()

,id,created_at,text,favorite_count,retweet_count
0,9.635508e+17,Tue Feb 13 23:09:37 +0000 2018,@cripto_miners Bitcoin Gold (BTG) is gaining p...,0.0,0.0
1,9.635508e+17,Tue Feb 13 23:09:38 +0000 2018,@Soc_Currency Successfully launch mainnet and ...,0.0,0.0
2,9.635508e+17,Tue Feb 13 23:09:38 +0000 2018,I don’t like #Ripple but I for sure am bullish...,0.0,0.0
3,9.635508e+17,Tue Feb 13 23:09:38 +0000 2018,RT @kucoincon_us: @kucoincom @covesting We're ...,0.0,0.0
4,9.635508e+17,Tue Feb 13 23:09:38 +0000 2018,New Airdrop https://t.co/CExaAFiCiN\n#Airdrop ...,0.0,0.0


In [4]:
df.shape

(6406528, 5)

In [6]:
# converting to date to pd date time
df['created_at'] = pd.to_datetime(df['created_at'])

In [7]:
type(df['created_at'])

pandas.core.series.Series

In [8]:
df.head()

,id,created_at,text,favorite_count,retweet_count
0,9.635508e+17,2018-02-13 23:09:37,@cripto_miners Bitcoin Gold (BTG) is gaining p...,0.0,0.0
1,9.635508e+17,2018-02-13 23:09:38,@Soc_Currency Successfully launch mainnet and ...,0.0,0.0
2,9.635508e+17,2018-02-13 23:09:38,I don’t like #Ripple but I for sure am bullish...,0.0,0.0
3,9.635508e+17,2018-02-13 23:09:38,RT @kucoincon_us: @kucoincom @covesting We're ...,0.0,0.0
4,9.635508e+17,2018-02-13 23:09:38,New Airdrop https://t.co/CExaAFiCiN\n#Airdrop ...,0.0,0.0


In [9]:
print(df.created_at.min())
print(df.created_at.max())

2018-02-13 23:09:37
2018-03-03 08:42:41


In [10]:
mask = (df['created_at'] > '2018-02-26 02:55:00') & (df['created_at'] <= '2018-03-03 08:43:41')
# print(df.loc[mask])

In [11]:
masked = df.loc[mask]

In [12]:
print(masked.created_at.min())
print(masked.created_at.max())

2018-02-27 05:06:02
2018-03-03 08:42:41


In [13]:
masked.shape

(2067946, 5)

In [14]:
masked.to_msgpack("full_DataFrame/combined_data/master/data_tweets_min.msg")

In [15]:
# masked['RT'] = np.where(masked['id'], 0, 0)

In [16]:
masked = masked.reset_index()

In [17]:
type(masked['text'][0])

str

In [ ]:
# if "RT" in masked['text'][1]:
#     print("1")

In [18]:
# df = pd.DataFrame(["BULL","BEAR","BULL"], columns=['A'])
masked["RT"] = [1 if "RT" in ele else 0 for ele in masked["text"]]

In [18]:
masked.set_index('created_at', inplace=True)

In [19]:
masked.head()

,id,text,favorite_count,retweet_count,RT
created_at,,,,,
2018-02-27 05:06:02,9.683515e+17,BITCOIN! Gold Plated Physical Bitcoin Collecto...,0.0,0.0,0
2018-02-27 05:06:03,9.683515e+17,RT @velixID: Here are the key features of #Vel...,0.0,0.0,1
2018-02-27 05:06:03,9.683515e+17,I think that #smokenetwork is going to take ov...,0.0,0.0,0
2018-02-27 05:06:04,9.683515e+17,RT @BitcoinWSpectrm: AIRDROP ALERT\nWe are giv...,0.0,0.0,1
2018-02-27 05:06:04,9.683515e+17,Fake Satoshi Accused of Stealing $10bn in BTC ...,0.0,0.0,0


In [20]:
def f(x):
     return pd.Series(dict(retweet_counts = x['RT'].sum(),
                        text = "{%s}" % ' '.join(x['text'])))

text_df = masked.groupby(pd.TimeGrouper('2Min')).apply(f)

In [21]:
text_df.head()

,retweet_counts,text
created_at,,
2018-02-27 05:06:00,285,{BITCOIN! Gold Plated Physical Bitcoin Collect...
2018-02-27 05:08:00,280,"{RT @MalwareJake: Dear web developers,\nThe an..."
2018-02-27 05:10:00,312,{@Crypto_Bossman My favorite cryptocurrency ex...
2018-02-27 05:12:00,280,{I should have asked @VitalikButerin why use E...
2018-02-27 05:14:00,317,"{RT @ViceToken: Why Watch Porn for Free, when ..."


In [22]:
text_df2 = masked.groupby(pd.TimeGrouper('5Min')).apply(f)

In [23]:
print(len(text_df))
print(len(text_df2))

2989
1196


In [24]:
text_df

,retweet_counts,text
created_at,,
2018-02-27 05:06:00,285,{BITCOIN! Gold Plated Physical Bitcoin Collect...
2018-02-27 05:08:00,280,"{RT @MalwareJake: Dear web developers,\nThe an..."
2018-02-27 05:10:00,312,{@Crypto_Bossman My favorite cryptocurrency ex...
2018-02-27 05:12:00,280,{I should have asked @VitalikButerin why use E...
2018-02-27 05:14:00,317,"{RT @ViceToken: Why Watch Porn for Free, when ..."
2018-02-27 05:16:00,276,"{RT @LivouMcoin: airdrop 5,000 LIVO/ETH addres..."
2018-02-27 05:18:00,265,{RT @maxkeiser: Poloniex deal signals “all cle...
2018-02-27 05:20:00,281,{Tech: Criminals are reportedly abducting the ...
2018-02-27 05:22:00,340,{RT @VentureCoinist: $BTC fractal comparing th...


In [25]:
text_df =text_df.reset_index()
text_df2 =text_df2.reset_index()

In [26]:
# text_df=text_df.drop(['favorite_count'], axis=1)
# text_df2=text_df2.drop(['favorite_count'], axis=1)

In [27]:
# newdf.get_group("2018-02-27 05:06:00")
text_df.head()

,created_at,retweet_counts,text
0,2018-02-27 05:06:00,285,{BITCOIN! Gold Plated Physical Bitcoin Collect...
1,2018-02-27 05:08:00,280,"{RT @MalwareJake: Dear web developers,\nThe an..."
2,2018-02-27 05:10:00,312,{@Crypto_Bossman My favorite cryptocurrency ex...
3,2018-02-27 05:12:00,280,{I should have asked @VitalikButerin why use E...
4,2018-02-27 05:14:00,317,"{RT @ViceToken: Why Watch Porn for Free, when ..."


In [28]:
text_df2.head()

,created_at,retweet_counts,text
0,2018-02-27 05:05:00,565,{BITCOIN! Gold Plated Physical Bitcoin Collect...
1,2018-02-27 05:10:00,768,{@Crypto_Bossman My favorite cryptocurrency ex...
2,2018-02-27 05:15:00,682,{RT @DocumentTailor: As the countdown continue...
3,2018-02-27 05:20:00,768,{Tech: Criminals are reportedly abducting the ...
4,2018-02-27 05:25:00,656,{@WandoViper Our $PRL luxuriousstar signal di...


In [29]:
text_df.to_msgpack("full_DataFrame/combined_data/master/grouped_text_2min.msg")
text_df2.to_msgpack("full_DataFrame/combined_data/master/grouped_text_5min.msg")

In [30]:
import re
from textblob import TextBlob

In [31]:
def clean_tweet(tweet):
    '''
    Utility function to clean tweet text by removing links, special characters
    using simple regex statements.
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

def get_tweet_sentiment(tweet):
    '''
    Utility function to classify sentiment of passed tweet
    using textblob's sentiment method
    '''
    # create TextBlob object of passed tweet text
    analysis = TextBlob(clean_tweet(tweet))
    # set sentiment
#         print(analysis.sentiment.polarity)
    return analysis.sentiment.polarity

In [32]:
tweets = []

#         try:
# call twitter api to fetch tweets
#             fetched_tweets = self.api.search(q = query, count = count)
#             print(fetched_tweets)
# parsing tweets one by one
#             for tweet in fetched_tweets:
for i in range(0,len(text_df)):
    # empty dictionary to store required params of a tweet
    parsed_tweet = {}

    # saving text of tweet
    parsed_tweet['text'] = text_df['text'][i]
    # saving sentiment of tweet
    parsed_tweet['sentiment'] = get_tweet_sentiment(parsed_tweet['text'])

    # appending parsed tweet to tweets list
#                 if tweet.retweet_count > 0:
#                     # if tweet has retweets, ensure that it is appended only once
#                     if parsed_tweet not in tweets:
#                         tweets.append(parsed_tweet)
#                 else:
#                     tweets.append(parsed_tweet)
    tweets.append(parsed_tweet)
# return parsed tweets

In [33]:
ptweets = [tweet for tweet in tweets if tweet['sentiment'] >0]
# percentage of positive tweets
print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets)))
# picking negative tweets from tweets
ntweets = [tweet for tweet in tweets if tweet['sentiment'] < 0]
# percentage of negative tweets
print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets)))
# percentage of neutral tweets
print("Neutral tweets percentage: {} %".format(100*(len(tweets) - len(ntweets) - len(ptweets))/len(tweets)))

# printing first 5 positive tweets
# print("\n\nPositive tweets:")
# for tweet in ptweets[:10]:
#     print(tweet['text'])

# printing first 5 negative tweets
# print("\n\nNegative tweets:")
for tweet in tweets[:10]:
    print(tweet['sentiment'])

Positive tweets percentage: 96.78822348611575 %
Negative tweets percentage: 0.06691201070592172 %
Neutral tweets percentage: 3.1448645031783204 %
0.18907832120598084
0.1992014254514255
0.1898770243398676
0.1940412700904929
0.15553662044945726
0.1720664334649711
0.18568431251288403
0.18259780206543444
0.19814220876720884
0.20113685064935066


In [34]:
tweets2 = []

#         try:
# call twitter api to fetch tweets
#             fetched_tweets = self.api.search(q = query, count = count)
#             print(fetched_tweets)
# parsing tweets one by one
#             for tweet in fetched_tweets:
for i in range(0,len(text_df2)):
    # empty dictionary to store required params of a tweet
    parsed_tweet = {}

    # saving text of tweet
    parsed_tweet['text'] = text_df2['text'][i]
    # saving sentiment of tweet
    parsed_tweet['sentiment'] = get_tweet_sentiment(parsed_tweet['text'])

    # appending parsed tweet to tweets list
#                 if tweet.retweet_count > 0:
#                     # if tweet has retweets, ensure that it is appended only once
#                     if parsed_tweet not in tweets:
#                         tweets.append(parsed_tweet)
#                 else:
#                     tweets.append(parsed_tweet)
    tweets2.append(parsed_tweet)
# return parsed tweets

In [35]:
ptweets = [tweet for tweet in tweets2 if tweet['sentiment'] >0]
# percentage of positive tweets
print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets)))
# picking negative tweets from tweets
ntweets = [tweet for tweet in tweets2 if tweet['sentiment'] < 0]
# percentage of negative tweets
print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets)))
# percentage of neutral tweets
print("Neutral tweets percentage: {} %".format(100*(len(tweets) - len(ntweets) - len(ptweets))/len(tweets)))

# printing first 5 positive tweets
# print("\n\nPositive tweets:")
# for tweet in ptweets[:10]:
#     print(tweet['text'])

# printing first 5 negative tweets
# print("\n\nNegative tweets:")
for tweet in tweets2[:10]:
    print(tweet['sentiment'])

Positive tweets percentage: 38.742054198728674 %
Negative tweets percentage: 0.03345600535296086 %
Neutral tweets percentage: 61.224489795918366 %
0.19392468719409542
0.1732009316219838
0.18581185782727594
0.1911957696498512
0.1993975386297766
0.1884411022669085
0.15009931345810174
0.17667774758013524
0.18369090701108515
0.19492949853271474


In [36]:
sen = pd.DataFrame(tweets)
sen2 = pd.DataFrame(tweets2)

In [37]:
sen.to_msgpack('full_DataFrame/combined_data/master/sentiment_data_2min.msg')
sen2.to_msgpack('full_DataFrame/combined_data/master/sentiment_data_5min.msg')

In [38]:
sen.head()

,sentiment,text
0,0.189078,{BITCOIN! Gold Plated Physical Bitcoin Collect...
1,0.199201,"{RT @MalwareJake: Dear web developers,\nThe an..."
2,0.189877,{@Crypto_Bossman My favorite cryptocurrency ex...
3,0.194041,{I should have asked @VitalikButerin why use E...
4,0.155537,"{RT @ViceToken: Why Watch Porn for Free, when ..."


In [39]:
sen2.head()

,sentiment,text
0,0.193925,{BITCOIN! Gold Plated Physical Bitcoin Collect...
1,0.173201,{@Crypto_Bossman My favorite cryptocurrency ex...
2,0.185812,{RT @DocumentTailor: As the countdown continue...
3,0.191196,{Tech: Criminals are reportedly abducting the ...
4,0.199398,{@WandoViper Our $PRL luxuriousstar signal di...


In [40]:
group_by_size = masked.groupby(pd.TimeGrouper('2Min')).size()
group_by_size2 = masked.groupby(pd.TimeGrouper('5Min')).size()
group_by_size.head()

created_at
2018-02-27 05:06:00    504
2018-02-27 05:08:00    513
2018-02-27 05:10:00    562
2018-02-27 05:12:00    536
2018-02-27 05:14:00    573
Freq: 2T, dtype: int64

In [41]:
group_by_size2.head()

created_at
2018-02-27 05:05:00    1017
2018-02-27 05:10:00    1405
2018-02-27 05:15:00    1277
2018-02-27 05:20:00    1371
2018-02-27 05:25:00    1230
Freq: 5T, dtype: int64

In [42]:
tweets_in_group = []
for i in range(0, len(group_by_size)):
    tweets_in_group.append(group_by_size[i])

In [43]:
tweets_in_group2 = []
for i in range(0, len(group_by_size2)):
    tweets_in_group2.append(group_by_size2[i])

In [44]:
tweets_in_group_df = pd.DataFrame(tweets_in_group, columns =[['no_tweets']])
tweets_in_group_df2 = pd.DataFrame(tweets_in_group2, columns =[['no_tweets']])

In [45]:
dat1 = text_df[['created_at', 'text']]
dat2 = sen[['sentiment']]
dat3 = tweets_in_group_df[['no_tweets']]
sen_data = dat1.join(dat2).join(dat3)
sen_data.head()

,created_at,text,sentiment,no_tweets
0,2018-02-27 05:06:00,{BITCOIN! Gold Plated Physical Bitcoin Collect...,0.189078,504
1,2018-02-27 05:08:00,"{RT @MalwareJake: Dear web developers,\nThe an...",0.199201,513
2,2018-02-27 05:10:00,{@Crypto_Bossman My favorite cryptocurrency ex...,0.189877,562
3,2018-02-27 05:12:00,{I should have asked @VitalikButerin why use E...,0.194041,536
4,2018-02-27 05:14:00,"{RT @ViceToken: Why Watch Porn for Free, when ...",0.155537,573


In [46]:
dat1 = text_df2[['created_at', 'text']]
dat2 = sen2[['sentiment']]
dat3 = tweets_in_group_df2[['no_tweets']]
sen_data2 = dat1.join(dat2).join(dat3)
sen_data2.head()

,created_at,text,sentiment,no_tweets
0,2018-02-27 05:05:00,{BITCOIN! Gold Plated Physical Bitcoin Collect...,0.193925,1017
1,2018-02-27 05:10:00,{@Crypto_Bossman My favorite cryptocurrency ex...,0.173201,1405
2,2018-02-27 05:15:00,{RT @DocumentTailor: As the countdown continue...,0.185812,1277
3,2018-02-27 05:20:00,{Tech: Criminals are reportedly abducting the ...,0.191196,1371
4,2018-02-27 05:25:00,{@WandoViper Our $PRL luxuriousstar signal di...,0.199398,1230


In [47]:
# sen_data.to_msgpack('full_DataFrame/combined_data/master/sentiment_data_2min.msg')
# sen_data2.to_msgpack('full_DataFrame/combined_data/master/sentiment_data_5min.msg')

sen_dat = pd.read_msgpack('full_DataFrame/combined_data/master/sentiment_data_2min.msg')
sen_data2 = pd.read_msgpack('full_DataFrame/combined_data/master/sentiment_data_5min.msg')

In [48]:
text_df2.head()

,created_at,retweet_counts,text
0,2018-02-27 05:05:00,565,{BITCOIN! Gold Plated Physical Bitcoin Collect...
1,2018-02-27 05:10:00,768,{@Crypto_Bossman My favorite cryptocurrency ex...
2,2018-02-27 05:15:00,682,{RT @DocumentTailor: As the countdown continue...
3,2018-02-27 05:20:00,768,{Tech: Criminals are reportedly abducting the ...
4,2018-02-27 05:25:00,656,{@WandoViper Our $PRL luxuriousstar signal di...


In [49]:
# # sen_dat
# dat1 = sen_dat[['created_at', 'sentiment', 'no_tweets']]
# dat2 = text_df[['retweet_counts']]
# # dat3 = users_data_2min[['user_followers_count', 'user_friends_count', 'user_verified', 'no_users']]
# data_2min = dat1.join(dat2)
# data_2min.to_msgpack('full_DataFrame/combined_data/master/sen_data_without_text_2min.msg')
# data_2min.head()

,created_at,sentiment,no_tweets,retweet_counts
0,2018-02-27 05:06:00,0.189078,504,285
1,2018-02-27 05:08:00,0.199201,513,280
2,2018-02-27 05:10:00,0.189877,562,312
3,2018-02-27 05:12:00,0.194041,536,280
4,2018-02-27 05:14:00,0.155537,573,317


In [50]:
# # sen_dat
# dat1 = sen_data2[['created_at', 'sentiment', 'no_tweets']]
# dat2 = text_df2[['retweet_counts']]
# # dat3 = users_data_2min[['user_followers_count', 'user_friends_count', 'user_verified', 'no_users']]
# data_5min = dat1.join(dat2)
# data_5min.to_msgpack('full_DataFrame/combined_data/master/sen_data_without_text_5min.msg')
# data_5min.head()

,created_at,sentiment,no_tweets,retweet_counts
0,2018-02-27 05:05:00,0.193925,1017,565
1,2018-02-27 05:10:00,0.173201,1405,768
2,2018-02-27 05:15:00,0.185812,1277,682
3,2018-02-27 05:20:00,0.191196,1371,768
4,2018-02-27 05:25:00,0.199398,1230,656


In [51]:
sen_data_without_text = sen_data[['created_at', 'sentiment', 'no_tweets']]
sen_data_without_text.head()

,created_at,sentiment,no_tweets
0,2018-02-27 05:06:00,0.189078,504
1,2018-02-27 05:08:00,0.199201,513
2,2018-02-27 05:10:00,0.189877,562
3,2018-02-27 05:12:00,0.194041,536
4,2018-02-27 05:14:00,0.155537,573


In [52]:
sen_data_without_text2 = sen_data2[['created_at', 'sentiment', 'no_tweets']]
sen_data_without_text2.head()

,created_at,sentiment,no_tweets
0,2018-02-27 05:05:00,0.193925,1017
1,2018-02-27 05:10:00,0.173201,1405
2,2018-02-27 05:15:00,0.185812,1277
3,2018-02-27 05:20:00,0.191196,1371
4,2018-02-27 05:25:00,0.199398,1230


In [53]:
sen_data_without_text.to_msgpack('full_DataFrame/combined_data/master/sen_data_without_text_2min.msg')
sen_data_without_text2.to_msgpack('full_DataFrame/combined_data/master/sen_data_without_text_5min.msg')